# **Use the stopwords file from after section 8 to proceed with model building**

# Importing the relevant libraries

In [45]:
import tensorflow
import os
import pandas as pd
import numpy as np
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


### Define file path & prepare the Drive and current working directory

In [46]:
file_input = '1_input_data.xlsx'
file_input_translated = "input_data_translated.xlsx"
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/'
os.chdir( file_path )
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive'

### Read the input excel

In [ ]:
df= pd.read_excel(file_input)

In [ ]:
df.shape

(8500, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  8492 non-null   object
 1   Description        8499 non-null   object
 2   Caller             8500 non-null   object
 3   Assignment group   8500 non-null   object
dtypes: object(4)
memory usage: 265.8+ KB


# 1. Missing Values
### 1. Handle the missing values in the "Short Description" and "Description" field, 

### 2. Process the short description and description columns separately

### 3. Missing values in short Description can be removed if we consider only the short description as our independent variable

### 4. Missing values in Description, one way would be to fill it with the text available in the short description if available or drop the row if not

In [ ]:
df.isna().sum()

Short description    8
Description          1
Caller               0
Assignment group     0
dtype: int64

1a.  Dropping the 8 null values in Short description

In [ ]:
short_desc_idx = df[df['Short description'].isnull() == True].index.tolist()

In [ ]:
short_desc_idx

[2604, 3383, 3906, 3910, 3915, 3921, 3924, 4341]

In [ ]:
df = df.drop(short_desc_idx,axis=0)

In [ ]:
df.isna().sum()

Short description    0
Description          1
Caller               0
Assignment group     0
dtype: int64

In [ ]:
print(df[df['Description'].isnull() == True].index)

Int64Index([4395], dtype='int64')


1b.  Replacing the null Description with the Short description

In [ ]:
df.iloc[4395]['Description'] = df.iloc[4395]['Short description']
print(df.iloc[4395])

Short description    erp SID_39 password reset
Description          erp SID_39 password reset
Caller                       rqtmpjdb ohitelsg
Assignment group                         GRP_0
Name: 4403, dtype: object


In [ ]:
df[df['Description'] == 0].index

Int64Index([], dtype='int64')

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head()

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [ ]:
df.to_excel("testingInd.xlsx",index=True)

### 2. Duplicate values in Short description & Description
Find indices that have the Short description and Description fields the same, we will ignore these indices while concatenating

In [ ]:
duplicate_list = []
for l in range(len(df)):
  if df.iloc[l]['Description'] == df.iloc[l]['Short description']:
    # print(df.iloc[l]['Description'])
    # print(df.iloc[l]['Short description'])
    duplicate_list.append(l)

In [ ]:
duplicate_df = pd.DataFrame(duplicate_list)

In [ ]:
duplicate_df.to_pickle("duplicates.pkl")

In [ ]:
len(duplicate_list)

2862

We will use duplicate_list for the indices that should not be concatented (Short description + Description)

# 3. Standardize the encoding to utf-8

## Unable to encode to ANSI in Colab but possible in windows machine

In [ ]:
!pip install chardet    #library for detecting the encoding 

In [ ]:
import chardet

In [ ]:
idx_list_enc = [(x,chardet.detect(str(df.iloc[x]['Description']).encode())['encoding']) for x in range(len(df))]

In [ ]:
df_enc = pd.DataFrame(idx_list_enc,columns = ['index','encoding'])

In [ ]:
df_enc['encoding'].unique()

array(['ascii', 'utf-8', 'Windows-1254'], dtype=object)

In [ ]:
df_enc['encoding'].value_counts()

ascii           7674
utf-8            817
Windows-1254       1
Name: encoding, dtype: int64

In [ ]:
df_enc[df_enc['encoding'] == 'Windows-1254'].index

Int64Index([1344], dtype='int64')

In [ ]:
print(df.iloc[1344]['Description'])
print(df.iloc[1344]['Description'].encode('Windows-1254').decode('utf8'))

einlasten bei it-germany, da hierfÃ¼r bereits ein ersatz-pc verbaut ist und ein neuer bestellt wurde.
einlasten bei it-germany, da hierfür bereits ein ersatz-pc verbaut ist und ein neuer bestellt wurde.


In [ ]:
df['Description'][1344] = df['Description'][1344].encode('Windows-1254').decode('utf8')
df['Short description'][1344] = df['Short description'][1344].encode('Windows-1254').decode('utf8')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.iloc[1344]

index                                                             1344
Short description    pc an r241 in halle c nähe büro nesner fährt n...
Description          einlasten bei it-germany, da hierfür bereits e...
Caller                                               dtrvxiuq bwuqdtfo
Assignment group                                                GRP_33
Name: 1344, dtype: object

In [ ]:
df.shape

(8492, 5)

**Run** **the below code on Windows machine**

*Problem 1 : Need to figure out a way to handle encoding from ANSI to decoding to utf-8*

In [ ]:
# for idx in range(len(df)):
#   df['Description'][idx] = df.iloc[idx]['Description'].encode('ANSI').decode('utf8')
#   df['Short description'][idx] = df.iloc[idx]['Short description'].encode('ANSI').decode('utf8')

**Importing the ansi encoded/utf decoded text from local machine**

In [ ]:
df = pd.read_excel("ansi_encoded_utf8_decoded.xlsx",index_col=0)

In [ ]:
df_inbetween = pd.read_excel("ansi_encoded_utf8_decoded_v1.xlsx",index_col=0)

In [ ]:
utf = [1554,3643,3645,3674,4876,5456,5819,5827]

In [ ]:
for c in utf:
  df['Short description'][c] = df_inbetween['Short description'][c]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for c in utf:
 print(df['Short description'][c])

答复: ticket_no1559458 : wireless guest access- hr_tool trainer
答复: 答复: order products online problem
答复: 答复: order products online problem
erro programdntya docad
答复: shipments iak
答复: 35969737/2032252
答复: ticket_no1538811::company center authorization
答复: ticket_no1538811::company center authorization


In [ ]:
df.to_excel("ansi_encoded_utf8_decoded_v2.xlsx")

In [ ]:
df = pd.read_excel("ansi_encoded_utf8_decoded_v3.xlsx",index_col=0)

In [ ]:
df = pd.read_excel("ansi_encoded_utf8_decoded_v4.xlsx",index_col=0)

In [ ]:
df.shape

(8492, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8492 entries, 0 to 8491
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  8492 non-null   object
 1   Description        8492 non-null   object
 2   Caller             8492 non-null   object
 3   Assignment group   8492 non-null   object
dtypes: object(4)
memory usage: 331.7+ KB


# 4. Translating the other languages to English using Googletrans

In [ ]:
import time

In [ ]:
!pip install googletrans

In [ ]:
from googletrans import Translator
translator = Translator()

**This code will run for almost 5-6 hours due to data size**

In [ ]:
# count=0
# non_English = []
# for t in range(len(df)):
#   raw_text_desc = df['Description'][t]
#   flag_desc = translator.detect(raw_text_desc)
#   time.sleep(7)
#   if flag_desc.lang != 'en':
#     translation = translator.translate(raw_text_desc,dest='en')
#     non_English.append(t)
#     count+=1
#     df['Description'][t] = translation.text
#     print("language is : {} , confidence level is  : {}, index is : {} , text is : {}".format(flag_desc.lang,flag_desc.confidence,t,translation.text))
#   else:
#     print("language is : {} , confidence level is  : {}, index is : {}".format(flag_desc.lang,flag_desc.confidence,t))
# print("count : {}".format(count))

In [ ]:
# non_English_df = pd.DataFrame(non_English)

In [ ]:
# non_English_df

In [ ]:
# non_English_df.to_pickle("Non_English_Indices.pkl")

**This wasn't run**

In [ ]:
count=0
non_English_sd = []
for t in range(7960,len(df)):
  raw_text_sd = df['Short description'][t]
  #print(raw_text)
  flag_sd_desc = translator.detect(raw_text_sd)
  time.sleep(7)
  if flag_sd_desc.lang != 'en':
    translation = translator.translate(raw_text_sd,dest='en')
    count+=1
    df['Short description'][t] = translation.text
    non_English_sd.append(t)
    print("language is : {} , confidence level is  : {}, index is : {} , text is : {}".format(flag_sd_desc.lang,flag_sd_desc.confidence,t,translation.text))
  else:
    print("language is : {} , confidence level is  : {}, index is : {}".format(flag_sd_desc.lang,flag_sd_desc.confidence,t))
print("count : {}".format(count))

language is : en , confidence level is  : 1.0, index is : 7960


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


language is : zh-CN , confidence level is  : 1.0, index is : 7961 , text is : The online system provided by the customer cannot be opened
language is : de , confidence level is  : 0.686801, index is : 7962 , text is : evening job in job scheduler: Job 1320
language is : en , confidence level is  : 1.0, index is : 7963
language is : en , confidence level is  : 0.98703134, index is : 7964
language is : en , confidence level is  : 1.0, index is : 7965
language is : en , confidence level is  : 1.0, index is : 7966
language is : en , confidence level is  : 1.0, index is : 7967
language is : en , confidence level is  : 1.0, index is : 7968
language is : en , confidence level is  : 1.0, index is : 7969
language is : en , confidence level is  : 0.98204625, index is : 7970
language is : en , confidence level is  : 1.0, index is : 7971
language is : en , confidence level is  : 1.0, index is : 7972
language is : en , confidence level is  : 1.0, index is : 7973
language is : en , confidence level 

In [ ]:
# non_English_sd = [164,186,222,223,255,265,,270,276,302,304,306,335,339,357,363,365,368,370,441,446,450,453,454,455,456,463,466,469,481,487,488,555,558,
# 559,565,570,572,574,578,582,586,589,595,608,610,618,3009]

In [ ]:
non_English_sd_df = pd.DataFrame(non_English_sd)

In [ ]:
non_English_sd_df.head(20)

,0
0,2260
1,2282
2,2285
3,2286
4,2301
5,2307
6,2354
7,2359
8,2382
9,2389


In [ ]:
non_English_sd_df.to_pickle("Non_English_sd_Indices_setTwelve.pkl")

In [ ]:
non_English_df = pd.DataFrame(non_English)

In [ ]:
# df = pd.read_excel("ansi_encoded_utf8_decoded.xlsx",index_col=0)

In [ ]:
df.shape

(8492, 5)

In [ ]:
print(df.iloc[1344]['Description'])
print(df.iloc[1344]['Description'].encode('Windows-1254').decode('utf8'))

einlasten bei it-germany, da hierfÃ¼r bereits ein ersatz-pc verbaut ist und ein neuer bestellt wurde.
einlasten bei it-germany, da hierfür bereits ein ersatz-pc verbaut ist und ein neuer bestellt wurde.


In [ ]:
df[df['Description'] == 0].index.tolist()

[]

In [ ]:
df['Description'][3954] = df['Short description'][3954]
df['Description'][6726] = df['Short description'][6726]
df['Description'][6880] = df['Short description'][6880]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df['Short description'][1554] = translator.translate(df['Short description'][1554],dest='en').text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['Short description'][1554]

'答复: ticket_no1559458 : wireless guest access- hr_tool trainer'

In [ ]:
joint_indices = pd.read_pickle("1_joint_indices.pkl")
short_desc_indices = pd.read_pickle("2_short_desc_indices.pkl")
desc_indices = pd.read_pickle("3_desc_indices.pkl")

In [ ]:
print(joint_indices.shape)
print(short_desc_indices.shape)
print(desc_indices.shape)

(640, 1)
(182, 1)
(101, 1)


In [ ]:
joint_indices = joint_indices[0].values.tolist()
short_desc_indices = short_desc_indices[0].values.tolist()
desc_indices = desc_indices[0].values.tolist()

In [ ]:
joint_indices.append(1344)

In [ ]:
print(len(joint_indices))
print(len(short_desc_indices))
print(len(desc_indices))

641
182
101


In [ ]:
for x in short_desc_indices:
  text = df.loc[x]['Short description']
  time.sleep(6)
  translation = translator.translate(text,dest='en')
  translated_text = translation.text
  df['Short description'][x] = translated_text
  print("index : {} , original text : {} , translated text : {}".format(x,text,translated_text))

index : 266 , original text : a kündigung for fgxprnub hlanwgqj, 11161827, effective 31.05.2017 has been approved.  , translated text : a kündigung for fgxprnub hlanwgqj, 11161827, effective 31.05.2017 has been approved.
index : 456 , original text : primary telephone flow down - company EU  - EU plant , translated text : primary telephone flow down - company EU - EU plant
index : 524 , original text : a kündigung for eluvxqhw gpbfkqeu, 11166428, effective 28.02.2017 has been approved.  , translated text : a kündigung for eluvxqhw gpbfkqeu, 11166428, effective 28.02.2017 has been approved.
index : 558 , original text : usa ou deletion , translated text : usa or deletion
index : 559 , original text : uacyltoe hxgaycze , translated text : uacyltoe hxgaycze
index : 565 , original text : new iphone activation /  die synchronisierung mit exchange activesync ist auf ihrem gerät vorübergehend blockiert, bis d , translated text : new iphone activation / the synchronization with exchange active

In [ ]:
for x in desc_indices:
  text = df['Description'][x]
  time.sleep(6)
  translation = translator.translate(text,dest='en')
  translated_text = translation.text
  df['Description'][x] = translated_text
  print("index : {} , original text : {} , translated text : {}".format(x,text,translated_text))

index : 4 , original text : skype error  , translated text : skype error
index : 124 , original text : from: tvcdfqgp nrbcqwgj 
sent: friday, october 28, 2016 3:13 pm
to: nwfodmhc exurcwkm
subject:  se ha bloqueado en forma temporal la sincronización de su dispositivo móvil mediante exchange activesync hasta que su administrador autorice el acceso.

hi

i received this message and our local it expert has told me to open a ticket.

 , translated text : from: tvcdfqgp nrbcqwgj
sent: friday, october 28, 2016 3:13 pm
to: nwfodmhc exurcwkm
subject: Your mobile device has been temporarily blocked from syncing via exchange activesync until your administrator authorizes access.

hi

i received this message and our local it expert has told me to open a ticket.
index : 251 , original text : bitte passwort für fygrwuna gomcekzi e-mail zurücksetzen
bitte neues passwort zu cwrikael.oanmsecr@gmail.com (manager)  , translated text : please reset password for fygrwuna gomcekzi e-mail
please new passwo

In [ ]:
for x in joint_indices:
  text_sd = df['Short description'][x]
  text_desc = df['Description'][x]
  time.sleep(6)
  translation_sd = translator.translate(text_sd,dest='en')
  translation_desc = translator.translate(text_desc,dest='en')
  df['Short description'][x] = translation_sd.text
  df['Description'][x] = translation_desc.text
  print("index : {} , text1 : {} , text2 : {}".format(x,df['Short description'][x],df['Description'][x]))

index : 164 , text1 : Withdrawn: ticket_no1564867 - comments added , text2 : received from: abcdri@company.com

windy shi will withdraw the email "ticket_no1564867 - comments added".


This message is for the use of designated recipients only, and it may contain content that is licensed, confidential and prohibited from disclosure under applicable laws. Except for the designated recipient, it is strictly forbidden for anyone else to spread, distribute or copy this message. If you have received this message in error, please notify the sender and delete the message.
[
select the following link to view the disclaimer in an alternate language.
index : 186 , text1 : Qingdao Xinghe Electromechanical Shipment Notification Email Setting , text2 : from:
sent: friday, october 28, 2016 7:20 am
to: nwfodmhc exurcwkm
subject: re: Qingdao Xinghe Electromechanical Shipment Notification Email Setting

dear,
pls help to update customer 4563729890 shipment notification email address: abcdegy@gmail.com



In [ ]:
df.iloc[8282]

index                                                             8290
Short description               abended job in job_scheduler: Job_1315
Description          received from: monitoring_tool@company.com\r\n...
Caller                                               ZkBogxib QsEJzdZO
Assignment group                                                 GRP_9
Name: 8282, dtype: object

In [ ]:
abended_list = [6372,7106,7168,7186,7187,7188,7220,7233,7260,7313,7377,7447,7448,7460,7468,7470,7559,
7583,7587,7591,7593,7598,7601,7603,7605,7608,7634,7648,7714,7732,7733,7734,7735,7736,7738,7739,7823,7927,7954,
8107,8244,8266,8269,8272,8276,8282,8284,8287,8290,8293,8299,8301,8303,8310,8311,8424]
abended_list_one = [7106,7168,7186,7187,7188,7220,7233,7260,7313,7377,7447,7448,7460,7468,7470,7559,7583,7587,7591,7593,7598,
7601,7603,7605,7608,7634,7648,7714,7732,7733,7734,7735,7736,7738,7739,7823,7927,7954,8107,8244,8266,8269,
8272,8276]


In [ ]:
len(abended_list)

55

In [ ]:
len(abended_list_one)

44

In [ ]:
df['Short description'][7105]

'evening job in job scheduler: Job 1338'

In [ ]:
import re

In [ ]:
pattern_abended = re.compile("abended")

In [ ]:
for a in ab_list:
  if "abended" in df['Short description'][a] or "Abended" in df['Short description'][a]:
    df['Short description'][a] = pattern_abended.sub("evening",df['Short description'][a].lower())
    print("index : {} , translated text : {}".format(a,df['Short description'][a]))


In [ ]:
 df.iloc[8491]

index                                                             8499
Short description    Different programs cannot be opened on several...
Description          an mehreren pc`s lassen sich verschiedene prgr...
Caller                                               kqvbrspl jyzoklfx
Assignment group                                                GRP_49
Name: 8491, dtype: object

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


index : 4 , original text : skype error  , translated text : skype error
index : 124 , original text : from: tvcdfqgp nrbcqwgj 
sent: friday, october 28, 2016 3:13 pm
to: nwfodmhc exurcwkm
subject:  se ha bloqueado en forma temporal la sincronización de su dispositivo móvil mediante exchange activesync hasta que su administrador autorice el acceso.

hi

i received this message and our local it expert has told me to open a ticket.

 , translated text : from: tvcdfqgp nrbcqwgj
sent: friday, october 28, 2016 3:13 pm
to: nwfodmhc exurcwkm
subject: Your mobile device has been temporarily blocked from syncing via exchange activesync until your administrator authorizes access.

hi

i received this message and our local it expert has told me to open a ticket.
index : 251 , original text : bitte passwort für fygrwuna gomcekzi e-mail zurücksetzen
bitte neues passwort zu cwrikael.oanmsecr@gmail.com (manager)  , translated text : please reset password for fygrwuna gomcekzi e-mail
please new passwo

In [ ]:
df.to_excel("afterDescbeforeJoint.xlsx")

In [ ]:
df = pd.read_excel("afterDescbeforeJoint.xlsx",index_col=0)

In [ ]:
df.shape

(8492, 5)

In [ ]:
df['Description'][1757] = df['Description'][1757].text
df['Description'][6526] = df['Description'][6526].text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
translator.translate(df['Description'][1757],dest='en').text

'发件人: white, ben <ben.white@hr_tool.com>\n发送时间: 2016年10月11日 9:00:30\n收件人: eva li; dyhtuiel.hiyhugins@company.com; stefyty parkeyhrt\n抄送: haiwei.liang@hr_tool.com; patience, rob\n主题: re: company intepmov imjukbqhing plan \n \nhi eva,\ncan you work with your local it team to determine your desktop ip address and provide it to dyhtuiel and stefyty.\n\nhi dyhtuiel, stefyty,\neva is a miowvyrs qkspyrdm located in the apac office . please check that the routing is in place from her network to connect over the vpn to hr_tool ( 12.161.199.3   to   185.42.190.22 ).'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


index : 164 , text1 : Withdrawn: ticket_no1564867 - comments added , text2 : received from: abcdri@company.com

windy shi will withdraw the email "ticket_no1564867 - comments added".


This message is for the use of designated recipients only, and it may contain content that is licensed, confidential and prohibited from disclosure under applicable laws. Except for the designated recipient, it is strictly forbidden for anyone else to spread, distribute or copy this message. If you have received this message in error, please notify the sender and delete the message.
[
select the following link to view the disclaimer in an alternate language.
index : 186 , text1 : Qingdao Xinghe Electromechanical Shipment Notification Email Setting , text2 : from:
sent: friday, october 28, 2016 7:20 am
to: nwfodmhc exurcwkm
subject: re: Qingdao Xinghe Electromechanical Shipment Notification Email Setting

dear,
pls help to update customer 4563729890 shipment notification email address: abcdegy@gmail.com



In [ ]:
df.head(5)

,index,Short description,Description,Caller,Assignment group
0,0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [ ]:
df.to_excel("translated_ansi_encoded_utf8_decoded_v4.xlsx",index=True)

In [ ]:
df = pd.read_excel("translated_ansi_encoded_utf8_decoded_v4.xlsx",index_col=0)

# 5. HTML Parsing using html unescape

In [4]:
import re
import string
import html

In [65]:
df = pd.read_excel("translated_ansi_encoded_utf8_decoded_v4.xlsx",index_col=0)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8492 entries, 0 to 8491
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  8492 non-null   object
 1   Description        8492 non-null   object
 2   Caller             8492 non-null   object
 3   Assignment group   8492 non-null   object
dtypes: object(4)
memory usage: 331.7+ KB


In [67]:
df['Description'] = df['Description'].apply(html.unescape)
df['Short description'] = df['Short description'].apply(html.unescape)

In [ ]:
# df.to_excel("unescape.xlsx",index=True)

# 6. Pattern Search and Remove

**Creating the patterns that need to be removed, these should be applied in the sequence as below**



In [68]:
pattern_url = re.compile('https?://\S+|www\.\S+') # to remove urls
pattern_email = re.compile('\S+@\S+') # to remove email IDs
pattern_special = re.compile('[^0-9a-zA-Z\']') # to remove all special characters other than numbers,text and apostrophe
pattern_extra_whitespace = re.compile("\s+") # to remove extra whitespaces
pattern_digits = re.compile('[0-9]') # to remove digits

# Dates get removed due to number and special character removal

## *Problem 2 : Need to figure out a way to handle roman numerals*

Set of functions to remove unwanted characters

In [19]:
def removeURL(txt):
  if re.search(pattern_url, txt):
    txt = pattern_url.sub('',txt)
    return txt
  else:
    return txt 

In [20]:
def removeEmail(txt):
  if re.search(pattern_email, txt):
    txt = pattern_email.sub('',txt)
    return txt
  else:
    return txt 

In [21]:
def removeSpecial(txt):
  if re.search(pattern_special, txt):
    txt = pattern_special.sub(' ',txt)
    return txt
  else:
    return txt 

In [22]:
def removeExtraWhitespace(txt):
  if re.search(pattern_extra_whitespace, txt):
    txt = pattern_extra_whitespace.sub(' ',txt)
    return txt
  else:
    return txt 

In [23]:
def removeDigits(txt):
  if re.search(pattern_digits, txt):
    txt = pattern_digits.sub('',txt)
    return txt
  else:
    return txt 

Applying the above functions to the "Short description" and "Description" columns

In [69]:
df['Short description'] = df['Short description'].apply(removeURL)
df['Short description'] = df['Short description'].apply(removeEmail)
df['Short description'] = df['Short description'].apply(removeSpecial)
df['Short description'] = df['Short description'].apply(removeExtraWhitespace)
df['Short description'] = df['Short description'].apply(removeDigits)
df['Description'] = df['Description'].apply(removeURL)
df['Description'] = df['Description'].apply(removeEmail)
df['Description'] = df['Description'].apply(removeSpecial)
df['Description'] = df['Description'].apply(removeExtraWhitespace)
df['Description'] = df['Description'].apply(removeDigits)

In [15]:
# df.to_excel("cleaned_translated_ansi_encoded_utf8_decoded_v4.xlsx",index=True)

# 7. Expand Contractions

**The below list can be expanded to include more contractions**
Currently it has the ones available in the dataset only

The below list of functions are applied sequentially

In [70]:
contractionList = {"i'm":"i am","i m\s":"i am ","im":"i am","i've":"i have","i'd":"i would","i'll":"i will","can't":"cannot","couldn't":"could not",
"doesn't":"does not","didn't":"did not","don't": "do not","won't":"will not","wouldn't":"would not","shouldn't":"should not",
"it's": "it is","you're" : "you are","we're" : "we are","they're":"they are","isn't":"is not","that's":"that is","aren't":"are not",
"she's":"she is","here's":"here is","let's":"let us","ma'am":"maam","isn't":"is not","should've":"should have","wasn't":"was not",
"that'll":"that will","weren't":"were not","he's":"he is","we've":"we have","they've":"they have","there's":"there is","what's":"what is",
"how's":"how is","hasn't":"has not","that's":"that is","what's":"what is","let's":"let us","here's":"here is","wasn't":"was not",
"haven't":"have not","who's":"whose","weren't":"were not","hasn't":"has not","haven't":"have not"}

In [26]:
pattern_apostrophe = re.compile('\'')
pattern_space_before = re.compile('\s\'')

In [30]:
def removeSpaceBetweenContraction(txt):
  if re.search(pattern_space_before, txt):
    txt = pattern_space_before.sub('\'',txt)
    return txt
  else:
    return txt 

In [28]:
def removeApostrophe(txt):
  if re.search(pattern_apostrophe, txt):
    txt = pattern_apostrophe.sub('',txt)
    return txt
  else:
    return txt 

In [71]:
df['Short description'] = df['Short description'].apply(removeSpaceBetweenContraction)
df['Description'] = df['Description'].apply(removeSpaceBetweenContraction)

In [32]:
def expandContractions(txt):
   for key in contractionList.keys():
    if len(re.findall(key,txt))!=0:
      value = contractionList[key]
      txt = re.sub(key,value,txt)
      return txt
    else:
      return txt

In [72]:
df['Short description'] = df['Short description'].apply(expandContractions)
df['Description'] = df['Description'].apply(expandContractions)

In [73]:
df['Short description'] = df['Short description'].apply(removeApostrophe)
df['Description'] = df['Description'].apply(removeApostrophe)

In [35]:
def leftStrip(txt):
  return txt.lstrip()

In [36]:
def rightStrip(txt):
  return txt.rstrip()

In [74]:
df['Short description'] = df['Short description'].apply(leftStrip)
df['Description'] = df['Description'].apply(leftStrip)
df['Short description'] = df['Short description'].apply(rightStrip)
df['Description'] = df['Description'].apply(rightStrip)

# 8. Stopwords Removal

In [38]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.lang.en.stop_words import STOP_WORDS

Caller Names list from Caller column to remove from the dataset

In [82]:
caller_list = df['Caller'].unique().tolist()
# caller_name_list=[]
# for g in range(len(caller_list)):
#   caller_name_list.append(caller_list[g].split()[0])
#   caller_name_list.append(caller_list[g].split()[1])
print(len(caller_list))

2948


In [98]:
def replaceNonsenseWords(txt):
  for text in caller_list:
    if len(re.findall(text,txt))!=0:
      print(text)
      print(txt)
      txt = re.sub(text,'',txt)
      print(txt)
  return txt

In [91]:
df['Short description'] = df['Short description'].apply(replaceNonsenseWords)

ecwtrjnq jpecxuty
call for ecwtrjnq jpecxuty
call for 
ecwtrjnq jpecxuty
call for ecwtrjnq jpecxuty
call for 
yhmwxsqj ugnthxky
user yhmwxsqj ugnthxky having issues logging to outlook
user  having issues logging to outlook
wseacnvi azvixyqg
reset the password for wseacnvi azvixyqg on erp qa erp
reset the password for  on erp qa erp
hckvpary emxbpkwy
reset passwords for hckvpary emxbpkwy using password management tool password reset
reset passwords for  using password management tool password reset
ijeqpkrz nwtehsyx
unable to access password management tool id password manager ijeqpkrz nwtehsyx grauw
unable to access password management tool id password manager  grauw
arexjftu ohxdwngl
support for fa gstry arexjftu ohxdwngl
support for fa gstry 
ecwtrjnq jpecxuty
call for ecwtrjnq jpecxuty
call for 
fygrwuna gomcekzi
reset the password for fygrwuna gomcekzi on e mail
reset the password for  on e mail
laeusvjo fvaihgpx
problems with drive z laeusvjo fvaihgpx
problems with drive z 
hprdlb

In [92]:
# df.to_excel("nonsense_sd.xlsx",index=True)

In [95]:
df.iloc[2709]['Description']

'support for alicona jionmpsf wnkpzcmv'

In [99]:
df['Description'] = df['Description'].apply(replaceNonsenseWords)

ecwtrjnq jpecxuty
call for ecwtrjnq jpecxuty
call for 
dkmcfreg anwmfvlg
received from hi the printer printer is not working and needs a part replaced can you reroute the jobs in queue to printer printer wihuyjdo qpogfwkb has indicated that prqos needs a new part and it may not deliver for a few days so the inwarehouse tools will need to print on printer for now this needs to be taken care of today since the inwarehouse tools are printed and are picked up by an outside vendor at   pm in usa on a daily basis please contact dkmcfreg anwmfvlgenkataramdntyana if you have questions about the jobs in queue for today
received from hi the printer printer is not working and needs a part replaced can you reroute the jobs in queue to printer printer wihuyjdo qpogfwkb has indicated that prqos needs a new part and it may not deliver for a few days so the inwarehouse tools will need to print on printer for now this needs to be taken care of today since the inwarehouse tools are printed and are picke

In [100]:
df.to_excel('nonsense.xlsx',index=True)

In [101]:
def removeStopwords(txt):
  text = nlp(txt)
  row =[]
  for tk in text:
    text_details = nlp.vocab[tk.text]
    if tk.text and tk.text.strip():
      flag=False
    else:
      flag = True
    if text_details.is_stop == False and flag == False:
      row.append(tk.text)
  #print(row)
  return ' '.join(row)

In [102]:
df['Short description'] = df['Short description'].apply(removeStopwords)
df['Description'] = df['Description'].apply(removeStopwords)

In [103]:
df.to_excel("stopwords.xlsx",index=True)

**Checking all the misspelled words in the data**

**Below code related to pyenchant will not work in colab**

In [60]:
!pip install pyenchant
import enchant
from enchant.checker import SpellChecker
import numpy as np
from collections import defaultdict

In [51]:
df = pd.read_excel("stopwords.xlsx",index_col=0)

In [52]:
def createComboColumn(d):
    d['Description'] = np.where(d['Description'].isna(),d['Short description'],d['Description'])
    d['Short description'] = np.where(d['Short description'].isna(),'',d['Short description'])
    d['Combo'] = np.where((d['Short description'] != d['Description']),d['Short description'] + ' ' + d['Description'],d['Description'])
    return d

No difference between US and UK and words like analyse and analyze are both shortlisted

In [61]:
chkr_us = SpellChecker("en_US")
chkr_uk = SpellChecker("en_GB")

In [ ]:
error_dictionary = defaultdict()
def createMisspelledDictionary(series,chkr):
    idx=0
    for item in series:
        chkr.set_text(item)
        for err in chkr:
            if err.word in error_dictionary.keys():
                error_dictionary[err.word]+=1
            else:
                error_dictionary[err.word]=1
    return error_dictionary  

In [ ]:
crazy_words_dictionary_us = createMisspelledDictionary(df['Combo'],chkr_us)
#crazy_words_dictionary_uk = createMisspelledDictionary(df['Combo'],chkr_uk)

# The above words can be removed from the data with the exception of the Jargon words below

**Manually Extracted Words from misspelled list that are sensible and not incorrect**
# Problem 3 :  Could not find a way to do this in automated way

In [47]:
import pickle

In [49]:
jargon = pd.read_pickle("jargon.pkl")